# Setup

In [1]:
import geopandas as gpd
from os.path import basename
import pandas as pd
import pyproj
from shapely.geometry import Point, shape
import numpy as np
from sodapy import Socrata
import s3fs
from tqdm import tqdm
from datetime import datetime, timedelta
from typing import Generator
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
from zipfile import ZipFile
import plotly.express as px

In [2]:
# Rides
TOTAL_RIDERSHIP_TABLE = "6iiy-9s97" # https://data.cityofchicago.org/Transportation/CTA-Ridership-Daily-Boarding-Totals/6iiy-9s97
L_RIDERSHIP_TABLE = "5neh-572f" # https://data.cityofchicago.org/Transportation/CTA-Ridership-L-Station-Entries-Daily-Totals/5neh-572f
BUS_RIDERSHIP_TABLE = "jyb9-n7fm" # https://data.cityofchicago.org/Transportation/CTA-Ridership-Bus-Routes-Daily-Totals-by-Route/jyb9-n7fm
DIVVY_RIDERSHIP_TABLE = "fg6s-gzvg" # https://data.cityofchicago.org/Transportation/Divvy-Trips/fg6s-gzvg
DIVVY_SUB_RIDERSHIP_TABLE = "4am4-35ir" # https://data.cityofchicago.org/Transportation/Divvy-Trips-Subscriber-Only/4am4-35ir

# Stations
L_STATIONS_TABLE = "8pix-ypme" # https://data.cityofchicago.org/Transportation/CTA-System-Information-List-of-L-Stops/8pix-ypme
DIVVY_STATIONS_TABLE = "bbyy-e7gq" # https://data.cityofchicago.org/Transportation/Divvy-Bicycle-Stations/bbyy-e7gq
BUS_ROUTES_TABLE = "6uva-a5ei" # https://data.cityofchicago.org/Transportation/CTA-Bus-Routes/6uva-a5ei
BUS_STOPS_TABLE = "pxug-u72f" # https://data.cityofchicago.org/Transportation/CTA-Bus-Stops-Shapefile/pxug-u72f
BUILDINGS_TABLE = "syp8-uezg" # https://data.cityofchicago.org/Buildings/buildings/syp8-uezg

UNITED_CENTER = ((41,52,50,"N"), (87,40,27,"W")) # lat/lng
MCCORMICK_PLACE = ((41,51,7,"N"), (87,36,58,"W"))

YMD = "%Y-%m-%d"
DNC_START = "2024-08-19"
DNC_END = "2024-08-22"
ERAS_START = "2023-06-02"
ERAS_END = "2023-06-04"
# TODO!
# Using the Taylor Swift Eras tour as a stand-in for the DNC since transit data isn't updated to August yet.
DNC_START = ERAS_START
DNC_END = ERAS_END
DNC_START_ISO = datetime.strptime(DNC_START, YMD).isoformat()
DNC_END_ISO = datetime.strptime(DNC_END, YMD).replace(hour=23, minute=59, second=59).isoformat()


# I'm going to consider Phase 5 as the start point for the new normal.
# This is "Illinois Restored", gatherings of 50+, including festivals, etc.
COVID = {"TIER_3": "2021-01-02",
         "TIER_1": "2021-01-19",
         "PHASE_4": "2021-02-02",
         "PHASE_5": "2021-06-11"}

LOCAL_CRS = pyproj.CRS("EPSG:3435") # NAD83 / Illinois East (ftUS)
WORLD_CRS = pyproj.CRS("EPSG:4326") # WGS84


## Socrata API

In [3]:
client = Socrata(
        "data.cityofchicago.org",
        app_token=None,
        timeout=10
    )

In [4]:
def _soda_fix_coltypes(df: pd.DataFrame, resource, aliases=None):
    """
    Coerce pandas dtypes because SodaPy seems to return everything as strings.
    """
    # Iterate through columns because query may subset columns
    meta = client.get_metadata(resource)
    colnames = [c['fieldName'] for c in meta['columns']]
    coltypes = [c['dataTypeName'] for c in meta['columns']]
    coltypes = {c: ct for c,ct in zip(colnames, coltypes)}
    if aliases is not None:
        for kv in aliases.split(','):
            if ' AS ' in kv:
                k = kv.split(' AS ')[0].strip()
                v = kv.split(' AS ')[1].strip()
                # Imperfectly extract column name from univariate expressions
                # XXX: Assumes the function doesn't change the dtype
                #      If we dont want that assumption we should only alias
                #      if k exactly equals a column name ie no function
                c = next(filter(lambda c: c in k, colnames), None)
                if c is not None:
                    coltypes[v] = coltypes[c]

    for col in df.columns:
        if col not in coltypes.keys():
            continue # functions of columns might not preserve type!
        elif coltypes[col] == 'calendar_date':
            df[col] = pd.to_datetime(df[col])
        elif coltypes[col] == 'number':
            df[col] = pd.to_numeric(df[col])
    return df

def _soda_to_df(data: Generator, has_header=True):
    """
    Collect iterable of rows into a dataframe.
    """
    if has_header:
        header = list(next(data, {}).keys())
        df = pd.DataFrame.from_records(data, columns=header)
    else:
        df = pd.DataFrame.from_records(data)
    return df

def soda_get_all(resource, has_header=True, **params):
    """Wrapper for client.get_all"""
    if "limit" in params.keys():
        # Sodapy doesn't allow limit with get_all :(
        data = iter(client.get(resource, **params))
    else:
        data = client.get_all(resource, **params)
    return (_soda_to_df(data, has_header)
            .pipe(_soda_fix_coltypes, resource, params.get('select',None)))

## S3 Bucket API

In [5]:
s3 = s3fs.S3FileSystem(anon=True)

In [6]:
def get_bucket_paths() -> Generator:
    """
    Returns pairs of s3://filepaths and membername.csv in the bucket.
    
    Handles:
    - ignoring irrelevant files in the bucket, like the index.html page
    - ignoring irrelevant member files like system files in the ZIP files
    - including multiple valid CSVs per ZIP like multiple quarters
    - some station_ids are normalized and some aren't
    """
    s3_paths = [f"s3://{x}" for x in s3.glob('divvy-tripdata/*.zip')]
    csv_filter = lambda x: x.endswith('.csv') and 'MACOSX' not in x
    shp_filter = lambda x: x.endswith('.shp.zip') and 'MACOSX' not in x
    trip_filter = lambda x: 'trip' in basename(x.lower()) and csv_filter(x)
    station_filter = lambda x: 'station' in basename(x.lower()) and (csv_filter(x) or shp_filter(x))
    for s3_path in s3_paths:
        with s3.open(s3_path, mode='rb') as s3f:
            with ZipFile(s3f) as zf:
                station_path = filter(station_filter, zf.namelist())
                station_path = sorted(station_path, key=csv_filter)
                station_path = station_path.pop() if station_path else None
                for csv_path in filter(trip_filter, zf.namelist()):
                    yield (s3_path, csv_path, station_path)
                if not any(map(csv_filter, zf.namelist())):
                    print(f"WARNING: Did not find csv in {s3_path}")

def station_schema(df):
    """Helper function to unify schema drift."""
    return (
        df.rename(columns= lambda x: x.lower())
          .rename(columns = {
            "lat": "latitude",
            "long": "longitude",
            "id_list": "station_id",
            "id": "station_id",
            "online date": "online_date"
        }))

def trip_schema(df):
    """Helper function to unify schema drift."""
    return (
        df.rename(columns={
            '01 - Rental Details Rental ID': 'ride_id',
            '01 - Rental Details Local Start Time': 'start_time',
            '01 - Rental Details Local End Time': 'end_time', 
            '01 - Rental Details Bike ID': 'bike_id',
            '01 - Rental Details Duration In Seconds Uncapped': 'tripduration',
            '03 - Rental Start Station ID': 'start_station_id',
            '03 - Rental Start Station Name': 'start_station_name',
            '02 - Rental End Station ID': 'end_station_id',
            '02 - Rental End Station Name': 'end_station_name',
            'User Type': 'user_type',
            'Member Gender': 'gender',
            '05 - Member Details Member Birthday Year': 'birthyear'
            })
          .rename(columns= lambda x: x.lower())
          .rename(columns = {
            'from_lng': 'start_lng',
            'from_lat': 'start_lat',
            'to_lng': 'end_lng',
            'to_lat': 'end_lat',
            'trip_id': 'ride_id',
            'from_station_id': 'start_station_id',
            'to_station_id': 'end_station_id',
            'from_station_name': 'start_station_name',
            'to_station_name': 'end_station_name',
            'starttime': 'start_time',
            'stoptime': 'end_time',
            'stop_time': 'end_time',
            'started_at': 'start_time',
            'ended_at': 'end_time',
            'bikeid': 'bike_id',
            'tripduration': 'trip_duration',
            'usertype': 'user_type',
            'member_casual': 'user_type',
            'duration': 'trip_duration',
            'birthday': 'birthyear',
          }))

def read_trip_file(fp):
    """Helper function to unify schema drift."""
    df = pd.read_csv(fp).pipe(trip_schema)
    df['start_time'] = pd.to_datetime(df['start_time'], errors='coerce')
    df['end_time'] = pd.to_datetime(df['end_time'], errors='coerce')
    return df

def s3_bike_trips(min_year, max_year):
    rides = []
    for zip_path, trip_path, station_path in tqdm(get_bucket_paths()):
        pats = [r"(\d{4}).*-divvy-tripdata.zip", r"Divvy_.*Trips_(\d{4}).*.zip"]
        matches = filter(None, map(lambda z: re.search(z, zip_path), pats))
        year = int(next(map(lambda y: y.group(1), matches)))
        if min_year <= year and year <= max_year:
            with s3.open(zip_path, mode='rb') as s3f, ZipFile(s3f) as zf, zf.open(trip_path) as tripf:
                rides.append(read_trip_file(tripf))
    return pd.concat(rides, ignore_index=True)

def s3_bike_stations(fp):
    """Helper function to unify schema drift."""
    if fp.name.endswith(".csv"):
        df = pd.read_csv(fp).pipe(station_schema)
    else:
        df = gpd.read_file(fp).pipe(station_schema)
    keep_cols = ['station_id','name','latitude','longitude','geometry']
    return df.filter(keep_cols)

def s3_point_gdf(df, lng_col, lat_col, loc_col):
    """Helper function to compute projected geometries."""
    crs = df.crs if isinstance(df, gpd.GeoDataFrame) else WORLD_CRS
    if loc_col not in df.columns:
        df = df.assign(**{loc_col: gpd.points_from_xy(df[lng_col], df[lat_col], crs=crs)})
    return gpd.GeoDataFrame(df, geometry=loc_col, crs=crs)

def project_gdf(gdf, crs=LOCAL_CRS):
    return gdf.to_crs(crs)

# def geofilter(df, lng_col, lat_col, loc_col, point, dist):
#     """Helper function to filter rows within dist meters of point."""
#     gdf = create_geodf(df, lng_col, lat_col, loc_col)
#     point = gpd.GeoSeries([point], crs=WORLD_CRS).to_crs(gdf.crs)
#     return gdf[gdf.geometry.dwithin(point[0], dist)]

# def station_geofilter(trip_df, station_df, point, dist):
#     """Top-level function to sptially filter normalized dataframes."""
#     filtered_stations = geofilter(station_df, 'longitude', 'latitude', 'geometry', point, dist)[['station_id']]
#     df_from = trip_df.merge(filtered_stations, left_on='start_station_id', right_on='station_id')
#     df_to = trip_df.merge(filtered_stations, left_on='end_station_id', right_on='station_id')
#     return pd.concat([df_from, df_to], ignore_index=True).drop(columns=['station_id']).drop_duplicates('ride_id')
    
# def trip_geofilter(trip_df, station_df, point, dist):
#     """
#     Top-level function to spatially filter single denormalized dataframe.
#     Takes station_df as a last resort when denormalized trip_df is incomplete.
#     """
#     if 'start_lng' in trip_df.columns:
#         df_from = geofilter(trip_df, 'start_lng', 'start_lat', 'start_loc', point, dist)
#         df_to = geofilter(trip_df, 'end_lng', 'end_lat', 'end_loc', point, dist)
#         return pd.concat([df_from, df_to], ignore_index=True).drop_duplicates('ride_id')
#     else:
#         # Coerce the dtypes to match to avoid a warning.
#         # In the bad case when dtypes dont match, this should still return empty.
#         # In the good case when dtypes already match, it shouldn't affect outcome either.
#         trip_df['start_station_id'] = trip_df['start_station_id'].astype(station_df['station_id'].dtype)
#         trip_df['end_station_id'] = trip_df['end_station_id'].astype(station_df['station_id'].dtype)
#         return station_geofilter(trip_df, station_df, point, dist)
    
def meter_to_foot(x):
    return x * 3.281

def agg_bike_trips(rides):
    rides['start_date'] = rides['start_time'].dt.date
    rides['end_date'] = rides['end_time'].dt.date
    start_rides = rides.rename(columns=lambda x: x.replace('start_',''))[['station_id','date']]
    end_rides = rides.rename(columns=lambda x: x.replace('end_',''))[['station_id','date']]
    rides = pd.concat([start_rides, end_rides], ignore_index=True)
    rides = rides.value_counts(['station_id','date']).rename('rides').reset_index()
    rides['date'] = pd.to_datetime(rides['date'])
    return rides

# Panels

## Train Panel

In [7]:
train_stations = soda_get_all(L_STATIONS_TABLE, select="stop_id, direction_id, stop_name, station_name, map_id, location")

In [8]:
# train_stations['location'].apply(shape) # is not working!
train_stations['latitude'] = train_stations['location'].apply(lambda x: x['latitude'])
train_stations['longitude'] = train_stations['location'].apply(lambda x: x['longitude'])
train_stations['geometry'] = gpd.points_from_xy(train_stations['longitude'], train_stations['latitude'])
train_stations = train_stations.drop(columns=['location', 'latitude', 'longitude'])
train_stations = gpd.GeoDataFrame(train_stations, geometry='geometry',crs=WORLD_CRS)
# nb: Each train station is represented as two "stops" per station: one in each direction.
#     For our purposes, since we don't model the direction of travel, we will drop the redundant "stop".
train_stations = train_stations.drop_duplicates(['station_name','map_id','geometry'])

## Bus Panel

In [9]:
bus_routes =  soda_get_all(BUS_ROUTES_TABLE, select="the_geom, route, name")
bus_routes['geometry'] = bus_routes['the_geom'].apply(shape)
bus_routes = bus_routes.drop(columns='the_geom')
bus_routes = gpd.GeoDataFrame(bus_routes, geometry='geometry',crs=WORLD_CRS)

### Bus Stops

In [10]:
bus_stops = gpd.read_file("https://data.cityofchicago.org/download/pxug-u72f/application/x-zip-compressed",
                          columns=['ROUTESSTPG', 'geometry','PUBLIC_NAM'])

In [11]:
bus_stops['ROUTESSTPG'] = bus_stops['ROUTESSTPG'].str.split(',')
bus_stops = bus_stops.explode('ROUTESSTPG').rename(columns={'ROUTESSTPG':'route'})
# nb: Compared to train stations, bus stop pairs on opposite sides of the street
#     aren't AS CLEANLY paired in the dataset. Though we could spatially join them
#     as 1-nearest-neighbor if we really wanted.

## Metra Lines

Metra does not provide machine-readable ridership reports. They have bar graphs of weekly total ridership and monthly ridership by line.

https://metra.com/ridership-reports

TODO!

But actually the Regional Transit Authority does provide machine-readable monthly ridership by line.

https://rtams.org/media/datasets/metra-ridership

## Bike Panel

In [12]:
bike_stations = []
for zip_path, _, station_path in tqdm(filter(lambda x: x[2], get_bucket_paths())):
    with (s3.open(zip_path, mode='rb') as s3f, ZipFile(s3f) as zf, zf.open(station_path) as stationf):
        df = s3_bike_stations(stationf)
        gdf = s3_point_gdf(df, "longitude","latitude","geometry")
        bike_stations.append(gdf)

17it [01:05,  3.84s/it]


In [13]:
# # Some data is projected and some is not so we need to reproject to combine.
# from collections import Counter
# Counter([x.crs.to_authority() for x in bike_stations])
# # Actually only one of the files was projected so we will un-project everything.

In [14]:
bike_stations = [project_gdf(x, WORLD_CRS) for x in bike_stations]
# Now we can combine.
bike_stations = pd.concat(bike_stations, ignore_index=True).drop_duplicates()
bike_stations = bike_stations.drop(columns=['latitude', 'longitude'])

## Points of Interest

### United Center

In [15]:
def dms_to_decimal(degrees, minutes, seconds, direction):
    decimal = degrees + minutes / 60 + seconds / 3600
    if direction in ['S', 'W']:  # South and West should be negative
        decimal = -decimal
    return decimal

uc_xy = dms_to_decimal(*UNITED_CENTER[1]), dms_to_decimal(*UNITED_CENTER[0])
uc_xy = Point(*uc_xy) # lng/lat

In [16]:
uc_buildings = soda_get_all(BUILDINGS_TABLE, 
            where=f"within_circle(the_geom, {uc_xy.y}, {uc_xy.x}, 250)")
uc_buildings['geometry'] = uc_buildings['the_geom'].apply(shape)
uc_buildings = gpd.GeoDataFrame(uc_buildings, geometry='geometry', crs=WORLD_CRS)

In [17]:
# # Verifying this is the right building
# print(uc_buildings.bldg_name1)
# ax = uc_buildings.iloc[0:1].plot()
# uc_buildings.iloc[1:].plot(ax=ax,color='orange')

In [18]:
uc_building = uc_buildings[uc_buildings['bldg_name1'] == 'UNITED CENTER']

### McCormick Place

In [19]:
mp_xy = dms_to_decimal(*MCCORMICK_PLACE[1]), dms_to_decimal(*MCCORMICK_PLACE[0])
mp_xy = Point(*mp_xy) # lng/lat

In [20]:
mp_buildings = soda_get_all(BUILDINGS_TABLE, 
        where=f"within_circle(the_geom, {mp_xy.y}, {mp_xy.x}, 250)")
mp_buildings['geometry'] = mp_buildings['the_geom'].apply(shape)
mp_buildings = gpd.GeoDataFrame(mp_buildings, geometry='geometry', crs=WORLD_CRS)

In [21]:
# # Verifying this is the right building
# print(mp_buildings.bldg_name1)
# ax = mp_buildings.iloc[0:1].plot()
# mp_buildings.iloc[1:].plot(ax=ax,color='orange')

In [22]:
mp_building = mp_buildings[mp_buildings['bldg_name1'] == 'HYATT REGENCY MCCORMICK PLACE']

### Airports

In [23]:
train_aiports = (train_stations.station_name == "O'Hare")|(train_stations.station_name == "Midway")

In [24]:
# nb: CTA busses don't go directly into O'Hare, nor even to the adjacent Mixed Modal Transit center.
bus_airports = bus_stops['PUBLIC_NAM']=="Midway Orange Line Station"

### Code Distances

In [25]:
def code_buffers(df: pd.DataFrame, geom:gpd.GeoSeries, geom_prefix:str, dists:list[int]):
    building_proj = geom.geometry.to_crs(LOCAL_CRS)
    buffers = [building_proj.buffer(meter_to_foot(d)) for d in dists]
    df_proj = df.geometry.to_crs(LOCAL_CRS)
    codes = {f"{geom_prefix}_{d}": df_proj.intersects(b[0]) * 1.0 for d,b in zip(dists,buffers)}
    return df.assign(**codes)

In [26]:
train_stations = train_stations.pipe(code_buffers, uc_building, "uc", [400,800,1600])
bike_stations = bike_stations.pipe(code_buffers, uc_building, "uc", [400,800,1600])
bus_stops = bus_stops.pipe(code_buffers, uc_building, "uc", [400,800,1600])

train_stations = train_stations.pipe(code_buffers, mp_building, "mp", [400,800,1600])
bike_stations = bike_stations.pipe(code_buffers, mp_building, "mp", [400,800,1600])
bus_stops = bus_stops.pipe(code_buffers, mp_building, "mp", [400,800,1600])

train_stations['airport'] = train_aiports
bus_stops['airport'] = bus_airports

In [27]:
# We only have ridership by bus route, not bus stop, so to aggregate stops to routes,
# we'll compute the mean number of stops within the buffer per route.
buffer_cols = ['uc_400','uc_800','uc_1600','mp_400','mp_800','mp_1600','airport']
bus_stops_uc = bus_stops.groupby('route',as_index=False)[buffer_cols].mean()
bus_routes = bus_routes.merge(bus_stops_uc, how='left')

## Combine Panels

In [28]:
def combine_panels(bus=None, train=None, bike=None):
    if bus is not None:
        bus = bus.rename(columns={'route':'id'}).assign(transit='bus', tid="bus_"+bus['route'].astype(str))
    if train is not None:
        train = train.rename(columns={'map_id':'id'}).assign(transit='train', tid="train_"+train['map_id'].astype(str))
    if bike is not None:
        bike = bike.rename(columns={'station_id':'id'}).assign(transit='bike', tid="bike_"+bike['station_id'].astype(str))
    panel = pd.concat(filter(lambda x: x is not None and not x.empty, [bus,train,bike]), ignore_index=True, join='inner')
    panel = panel.drop(columns=['geometry'])
    return panel

In [29]:
transit_panel = combine_panels(bus_routes, train_stations, bike_stations)

# Naive Models

## Two-Period Pooled Panel

We estimate:

$$ rides_t = \beta_0 + \beta_1 \text{DNC}_t$$

with two weeks of data, where $\text{DNC}_t=1$ during the DNC week and 0 otherwise.
Note there is no error term because we have pooled over all units.

In [30]:
# nb: SoQL function "between X and Y" is not inclusive! so we'll use gte and lte

control_start_iso = (datetime.fromisoformat(DNC_START_ISO) - timedelta(days=7)).isoformat()
control_end_iso = (datetime.fromisoformat(DNC_END_ISO) - timedelta(days=7)).isoformat()

where_date = f"""(('{DNC_START_ISO}' <= service_date) AND (service_date <= '{DNC_END_ISO}'))
                OR (('{control_start_iso}' <= service_date) AND (service_date <= '{control_end_iso}'))"""
rides = soda_get_all(TOTAL_RIDERSHIP_TABLE, 
             has_header=True,
             select="total_rides,service_date", 
             where=where_date) \
        .assign(DNC = lambda x: x['service_date'] >= DNC_START_ISO)

model_data = rides.groupby('DNC')['total_rides'].sum()

In [31]:
# Computing the estimates by hand because we literallly only have two observations!
naive_pooled = {
    "beta_0" : model_data[False],
    "beta_1" : model_data[True] - model_data[False]
}
naive_pooled


{'beta_0': np.int64(1152467), 'beta_1': np.int64(937382)}

The above shows that there are alot more rides during the DNC week than the prior week.

## Two Period Panel

We estimate:

$$ rides_{it} = \beta_0 + \beta_1 \text{DNC}_t + u_i$$

with two weeks of data, where $\text{DNC}_t=1$ during the DNC week and 0 otherwise.
Note now we have an error term because we are allowing variation across units.

In [32]:
def get_rides_panel(ctl_start, ctl_end, trt_start, trt_end):
    where_date = f"""(('{trt_start}' <= date) AND (date <= '{trt_end}'))
                OR (('{ctl_start}' <= date) AND (date <= '{ctl_end}'))"""
    
    train_rides = soda_get_all(L_RIDERSHIP_TABLE, 
                            select="station_id,date,daytype,rides",
                            where=where_date) \
                .merge(train_stations, left_on='station_id', right_on='map_id')

    bus_rides = soda_get_all(BUS_RIDERSHIP_TABLE, 
                            select="route,date,daytype,rides",
                            where=where_date) \
                .merge(bus_routes, on='route')

    # bike_rides = s3_bike_trips(control_start.year, 
    #                         datetime.strptime(DNC_END, YMD).year) \
    #             .pipe(agg_bike_trips)
    
    # TODO! Add bike rides
    # bike_rides = bike_rides.merge(bike_stations, on='station_id')

    rides = combine_panels(bus_rides, train_rides, None)
    rides['DNC'] = (trt_start <= rides['date']) & (rides['date'] <= trt_end)
    return rides

In [33]:
rides = get_rides_panel(control_start_iso, control_end_iso, DNC_START_ISO, DNC_END_ISO)

In [34]:
# TODO! We are pooling daily observations into before and after time periods.
# I'd assumed the unaccounted for daily variation would be a problem, but
# aggregating it away ends up increasing our standard errors.
model_data = rides.groupby(['tid','DNC'], as_index=False)['rides'].sum()
naive_two_period = sm.OLS.from_formula("rides ~ DNC", model_data).fit()
print(naive_two_period.summary())

                            OLS Regression Results                            
Dep. Variable:                  rides   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.1619
Date:                Sat, 19 Oct 2024   Prob (F-statistic):              0.688
Time:                        12:09:37   Log-Likelihood:                -5561.2
No. Observations:                 533   AIC:                         1.113e+04
Df Residuals:                     531   BIC:                         1.113e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    7039.5075    505.201     13.934      

This shows that the DNC week does not significantly affect average rides systemwide.

In [35]:
# But the box plot confirms that the unit-level differences are not significant.
px.box(model_data, x='DNC', y='rides')

## Cross Section During DNC Week

We estimate:

$$ rides_i = \beta_0 + \beta_2 \text{UC}_i + u_i $$

during the DNC week, where $\text{UC}_i$=1 if the unit is within the United Center security perimeter.

In [36]:
# For starters lets look at the maximal version of this model:
rides['UCMP16'] = (rides['uc_1600'] > 0) | (rides['mp_1600'] > 0)
model_data = rides.groupby(['tid','DNC','UCMP16'], as_index=False)['rides'].sum()
naive_xs = sm.OLS.from_formula("rides ~ UCMP16", model_data, subset=model_data['DNC']).fit()
print(naive_xs.summary())

                            OLS Regression Results                            
Dep. Variable:                  rides   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     5.987
Date:                Sat, 19 Oct 2024   Prob (F-statistic):             0.0151
Time:                        12:09:39   Log-Likelihood:                -2795.1
No. Observations:                 267   AIC:                             5594.
Df Residuals:                     265   BIC:                             5601.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       6871.3080    555.241     12.

This shows that stations near the DNC on average may have had more rides than other stations.

In [37]:
# The box plot confirms that the means (at least) are marginally different
px.box(model_data[model_data['DNC']], x='UCMP16', y='rides')

Now let's look at a "full" design of all these things.

In [38]:
rides['UC4'] = (rides['uc_400'] > 0)
rides['UC8'] = (rides['uc_800'] > 0)
rides['UC16'] = (rides['uc_1600'] > 0)
rides['MP4'] = (rides['mp_400'] > 0)
rides['MP8'] = (rides['mp_800'] > 0)
rides['MP16'] = (rides['mp_1600'] > 0)
rides['UCMP4'] = (rides['uc_400'] > 0) | (rides['mp_400'] > 0)
rides['UCMP8'] = (rides['uc_800'] > 0) | (rides['mp_800'] > 0)
rides['UCMP16'] = (rides['uc_1600'] > 0) | (rides['mp_1600'] > 0)

# TODO! We are pooling daily observations into before and after time periods.
# I'd assumed the unaccounted for daily variation would be a problem, but
# aggregating it away ends up increasing our standard errors.
uc_cols = rides.filter(regex="UC|MP").columns
model_data = rides[rides['DNC']].groupby(['tid'] + list(uc_cols), as_index=False)['rides'].sum()
naive_uc4 = sm.OLS.from_formula("rides ~ UC4", model_data).fit()
naive_uc8 = sm.OLS.from_formula("rides ~ UC8", model_data).fit()
naive_uc16 = sm.OLS.from_formula("rides ~ UC16", model_data).fit()
naive_mp4 = sm.OLS.from_formula("rides ~ MP4", model_data).fit()
naive_mp8 = sm.OLS.from_formula("rides ~ MP8", model_data).fit()
naive_mp16 = sm.OLS.from_formula("rides ~ MP16", model_data).fit()
naive_ucmp4 = sm.OLS.from_formula("rides ~ UCMP4", model_data).fit()
naive_ucmp8 = sm.OLS.from_formula("rides ~ UCMP8", model_data).fit()
naive_ucmp16 = sm.OLS.from_formula("rides ~ UCMP16", model_data).fit()

from statsmodels.iolib.summary2 import summary_col
summary_col([naive_uc4,naive_uc8,naive_uc16,
             naive_mp4,naive_mp8,naive_mp16,
             naive_ucmp4,naive_ucmp8,naive_ucmp16])

,rides I,rides II,rides III,rides IIII,rides IIIII,rides IIIIII,rides IIIIIII,rides IIIIIIII,rides IIIIIIIII
Intercept,7245.0871,7194.9768,7126.1036,7221.7698,7169.0769,7099.1383,7138.3130,7028.6706,6871.3080
,(529.8843),(535.0636),(543.2671),(525.7363),(532.7218),(539.9935),(527.2787),(539.0041),(555.2406)
UC4[T.True],7265.2462,,,,,,,,
,(4998.9182),,,,,,,,
UC8[T.True],,4396.8982,,,,,,,
,,(3091.1215),,,,,,,
UC16[T.True],,,3347.7714,,,,,,
,,,(2219.2644),,,,,,
MP4[T.True],,,,14010.7302,,,,,
,,,,(6074.4727),,,,,


This shows that the stations near these places have more rides on average,
and that the stations sort of near these places have average or below average rides.
Moreover, the UC-area stations have more ridership than MP.

## Power Analysis

According to a Choose Chicago [report](https://cdn.choosechicago.com/uploads/2024/10/TE-DNC-Impact.pdf), the DNC attracted 50,000 visitors.

Given our sample size of ?? time periods and stations, desired power, and desired alpha,
what effect sizes can we predict?

Following [Bloom (1995)](https://journals.sagepub.com/doi/epdf/10.1177/0193841X9501900504):

the minimum detectable effect size is simply computed by comparing the one-sided
z score to reject the null hypothesis vs the one-sided z score to accept the alternate
hypothesis. i.e. for $\alpha=.05$ and $\beta=.8$ we have $MDE = 2.49\sigma_c$
whereas for $\alpha=.1$ and $\beta=.8$ we have $MDE = 2.12\sigma_c$ where $\sigma_c$
is the standard error of the estimate.

One can easily extend this analysis to two-sided cases. Bloom uses one-sided for its
simplicity in testing whether the *intended* effect happened or not. One-sided
tests have greater statistical power (produce smaller MDE's) than two-sided.
But in practice researchers do use the two-sided test because they can't theoretically rule out
the possibility of observing an unintended effect.

In [39]:
# # Bloom presents this formula for the standard error, noted as sigma_c
# sigma2 = rides['rides'].var()
# R2 = naive_two_period.rsquared
# T = rides['DNC'].mean()
# n = len(rides)
# sigma_c = np.sqrt(sigma2 * (1 - R2) / (T * (1 - T) * n))
zcritical = pd.Series({".1": 2.12, ".05": 2.49},name="zcritical")
# zcritical * sigma_c

# Conveniently, sigma_c == statsmodels.model.bse
def mde(model):
    return pd.concat(
        [pd.DataFrame(np.outer(model.bse, zcritical),
             columns="MDE at " + zcritical.index, 
             index=model.bse.index),
        pd.Series(model.params, name='estimated')], axis=1)

In [40]:
mde(naive_two_period)

,MDE at .1,MDE at .05,estimated
Intercept,1071.025755,1257.950061,7039.507519
DNC[T.True],1513.240264,1777.343517,287.211582


This shows we are severely underpowered to reject the null hypothesis if the
true effect sizes are indeed equal to these estimates.

In [41]:
mde(naive_xs)

,MDE at .1,MDE at .05,estimated
Intercept,1177.110090,1382.549115,6871.308017
UCMP16[T.True],3511.656969,4124.540496,4053.158650


This shows we almost have enough power if the true effect size is indeed as big as estimated.

# Panel Models

In all of these models Woldridge provides examples that only compare two time periods,
usually two (not necessarily consecutive) years. Somehow he doesn't say anything
about parallel trends in DiD, but it seems possible that it's common to use a two-period design.

For the rest of these models, we will use the 1600m buffer as our covariate, 
because even though the effect size
is most attenuated there, this gives us the largest treatment size. So this is 
actually the most conservative option and reduces variance in our errors.

## Diff in Diff

**Motivation**

TODO!

The simple difference-in-difference model combines the naive models with two weeks of data:

$$ rides_{it} = \beta_0 + \beta_1 \text{DNC}_t + \beta_2 \text{UC}_i + \beta_3 \text{DNC}_t \text{UC}_i  + u_{it} $$

where $\text{DNC}_t=1$ during the DNC week and 0 otherwise, and $\text{UC}_i=1$ if the station is inside the security perimeter and 0 otherwise.

In [42]:
# As you see below, there's barely any stops within the 400m buffer, so 
# this would be a terrible treatment sample.
pd.Series(rides.filter(regex=r"uc|mp").sum(), name="treat size")

uc_400      1.013125
uc_800     15.296226
uc_1600    38.952567
mp_400      0.286261
mp_800      8.011214
mp_1600    17.927111
Name: treat size, dtype: float64

In [43]:
# TODO! We are pooling daily observations into before and after time periods.
# I'd assumed the unaccounted for daily variation would be a problem, but
# aggregating it away ends up increasing our standard errors.
model_data = rides.groupby(['DNC','UCMP16','tid'],as_index=False)['rides'].sum()
did_model = sm.OLS.from_formula("rides ~ UCMP16 * DNC", model_data).fit()
print(did_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  rides   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     4.299
Date:                Sat, 19 Oct 2024   Prob (F-statistic):            0.00520
Time:                        12:09:40   Log-Likelihood:                -5554.9
No. Observations:                 533   AIC:                         1.112e+04
Df Residuals:                     529   BIC:                         1.113e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

This shows that the United Center and McCormick Place generally have more rides,
but that the DNC week didn't have significantly more rides, and UC and MP didn't
have significantly more rides during the DNC week.

### Threats

**Parallel Trends Assumption**

In [44]:
trend_start = datetime.strptime(DNC_START, YMD) - timedelta(days=31)
trend_start_iso = trend_start.isoformat()
trend_end = datetime.strptime(DNC_END, YMD).replace(hour=23,minute=59,second=59) - timedelta(days=7)
trend_end_iso = trend_end.isoformat()

rides_trend = get_rides_panel(trend_start_iso, trend_end_iso, DNC_START_ISO, DNC_END_ISO)
rides_trend['UCMP16'] = (rides_trend['uc_1600'] > 0) | (rides_trend['mp_1600'] > 0)

In [45]:
# It's hard to see whether there are parallel trends here... too many data points.
fig = px.line(rides_trend, x='date', y='rides', color='UCMP16')
fig.add_vline(x=trend_end.strftime(YMD), line_dash="dash", line_color="gray")
fig.add_vline(x=DNC_START, line_dash="dash", line_color="gray")
fig.show()

In [46]:
rides_trend_agg = rides_trend.groupby(['date','UCMP16'],as_index=False)['rides'].agg(['mean','min','max'])
rides_trend_agg = rides_trend_agg.melt(id_vars=['date','UCMP16'], value_vars=['mean','min','max'])

In [47]:
# You can sort of see there are parallel trends within each quantile.
# TODO! But I'm not sure if I'm accounting for quantile very well. Might need matched design.
fig = px.line(rides_trend_agg, x='date', y='value', color='UCMP16', line_dash='variable')
fig.add_vline(x=trend_end.strftime(YMD), line_dash="dash", line_color="gray")
fig.add_vline(x=DNC_START, line_dash="dash", line_color="gray")
fig.show()

**Stable Unit Treatment Value Assumption (SUTVA)**

There's definitely a possibility of SUTVA violation considering how ad-hoc
the spatial treatment areas are defined. If spillover is happening, then we
ought to see a positive DNC coef but an attenuated DNC*UC coef.

**Exog of Treatment**

This is definitely a possibility. The DiD doesn't fully account for this because
even though it separates out UC vs Non-UC at baseline, we actually expect UC to
be *more* able to accomodate surges in transit compared to other areas. So this
sounds like a threat to external validity.

## Fixed Effects

We estimate:

$$
\text{rides}_{it} = \beta_0 + \beta_1 \text{DNC}_t + \beta_2 \text{UC}_i + \gamma_i +  u_{it}
$$

In [51]:
model_data = rides.groupby(['DNC','UCMP16','tid'],as_index=False)['rides'].sum()
fe_model = sm.OLS.from_formula("rides ~ DNC + UCMP16 + C(tid)", model_data).fit()
print(fe_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  rides   R-squared:                       0.976
Model:                            OLS   Adj. R-squared:                  0.953
Method:                 Least Squares   F-statistic:                     41.19
Date:                Sat, 19 Oct 2024   Prob (F-statistic):          2.19e-139
Time:                        12:11:10   Log-Likelihood:                -4562.0
No. Observations:                 533   AIC:                             9660.
Df Residuals:                     265   BIC:                         1.081e+04
Df Model:                         267                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              6529.25

In [62]:
summary_col([naive_two_period, naive_xs, did_model, fe_model], 
            model_names=['Naive I','Naive II','DiD','FE'],
            stars=True,
            regressor_order=['Intercept',"DNC[T.True]","UCMP16[T.True]"], drop_omitted=True)

,Naive I,Naive II,DiD,FE
Intercept,7039.5075***,6871.3080***,6605.3671***,6529.2532***
,(505.2008),(555.2406),(529.8905),(112.7687)
DNC[T.True],287.2116,,265.9409,311.8534**
,(713.7926),,(749.3783),(155.1571)
UCMP16[T.True],,4053.1586**,3982.1157**,-6069.1066***
,,(1656.4420),(1604.8260),(1785.4214)
R-squared,0.0003,0.0221,0.0238,0.9765
R-squared Adj.,-0.0016,0.0184,0.0183,0.9528


In [72]:
constraint = " = ".join(fe_model.params.index[fe_model.params.index.str.startswith("C(tid)")]) + " = 0"
fe_model.f_test(constraint)

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=40.62514467864315, p=1.4393649376414982e-138, df_denom=265, df_num=266>

### Threats

TODO!

The this shows the R2 suddenly becomes extremely tight, due to finally controlling for station/line variance.

The DNC also finally becomes significant, but the UCMP (which was always sig) interestingly switches sign.

TODO!
This gives me three ideas:

* Use MLEM model to control for unit variance without using as many parameters
* Try simply controlling for transit type instead of having full idiosyncratic effects

## First Difference

**Motivation**

We want to eliminate the need to estimate these unobserved idiosyncratic time-invariant effects, $\gamma_i$.
Even though we can estimate their intercepts, they're not actually "observed".
Typically we consider $\gamma_i$ as a decomposition of the error term  $u_{it} = \gamma_i + e_{it}$,
therefore if $\gamma_i$ are correlated with X, this breaks $E(cov(X,u))=0$ assumption.

We estimate:

$$
\begin{aligned}
\text{rides}_{t+1} - \text{rides}_t &= \\
& \beta_0 + \beta_1 (\text{DNC}_1=1) + \beta_2 \text{UC}_i + \gamma_i +  u_{i1} \\
-& \beta_0 + \beta_1 (\text{DNC}_0=0) + \beta_2 \text{UC}_i + \gamma_i +  u_{i0} \\
\Delta \text{rides}_i &=  \beta_1 + \Delta u_i
\end{aligned}
$$


Unfortunately, this method will not work because we have not used X's that vary over time.
We cannot separate $\beta_1$ from the unobserved error terms.

In [93]:
# For kicks, let's run this model anyway ...
model_data = rides.groupby(['tid','UCMP16','DNC'], sort=True, as_index=False)['rides'].sum()
is_full_panel = model_data.tid.value_counts() == 2
model_data = model_data[model_data['tid'].isin(is_full_panel[is_full_panel].index)]
assert(all(model_data['tid'].value_counts() == 2))
model_data = model_data[model_data['DNC']].set_index('tid')[['rides','UCMP16']]*1.0 \
            - model_data[~model_data['DNC']].set_index('tid')[['rides','UCMP16']]*1.0
model_data = model_data.reset_index()
assert(all(model_data['UCMP16'] == 0))
fd_model = model_data['rides'].mean()
fd_model

np.float64(311.8533834586466)

This shows that the FD $\beta_1$ estimate reproduces the FE estimate. This is surprising.

I guess it's because we have two periods, which makes $\gamma_i = \bar{\text{rides}_i}$
equal to $\text{rides}_{i1}-\text{rides}_{i0}$

In [97]:
# Just curious how this is distributed.
# px.box(model_data['rides'])

### Threats

TODO!

Noting the box plot above, we may want to drop some very out-of-sample stations?
Or at least check a sensitivity analysis. In the non-delta case the range is a factor of 10
but in the delta case it is a factor of 100.

TODO!

Check the notes in the "More Panel Models" section because some apply to FD.

# Time Series

TODO!

Decide what time periods you want to include. This will significantly decrease
query times for bike and rail tables.

TODO!

Also decide the time granularity. Do you need it to be daily? Weekly may also make sense.
That may help smooth out the variation meaning you can use a simpler model. 
Otherwise a linear model will lose a lot of explanatory power comparing weekdays to weekends.
But maybe just add a weekday and also quarter or month fixed effect and plot the residuals of all the predicted vs real values.

In [73]:
ts = soda_get_all(TOTAL_RIDERSHIP_TABLE, 
             select="service_date AS date, total_rides AS rides")

In [74]:
ts['is_dnc'] = (DNC_START_ISO <= ts['date']) & (ts['date'] <= DNC_END_ISO)
ts = pd.concat([ts, ts.date.dt.isocalendar()], axis=1)
ts['yearweek'] = ts['year'].astype(str).str.cat(ts['week'].astype(str).str.pad(2,'left','0'),sep='-')
ts = ts.groupby('date',as_index=False).last()


In [ ]:
# The process looks stationary but after pandemic there is a growth factor.
# I think we should take only post pandemic data because otherwise we have to
# add a post-pandemic interaction to all the regression terms.
ts.plot(x='date',y='rides')

In [ ]:
# Re-plotting at weekly to reduce high-frequency variation.
# (Omitted: validated that the periodic drops are due to 
# the christmas holiday season. the iso transformation creates stable number of
# rows per week and per year.)
ts.groupby('yearweek')['rides'].sum().plot()

## TS Models

In [77]:
model_data = ts[ts.date > datetime.strptime(COVID['PHASE_5'], YMD)].groupby('yearweek',as_index=False)['rides'].sum()


### Seasonal

In [ ]:
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.seasonal import seasonal_decompose

# This is pretty bad. It may or may not be seasonal. 
# If it is seasonal, this is the wrong way to estimate it.
stl = STL(model_data['rides'], period=52)
res = stl.fit()
fig = res.plot()


# # Same with this.
# res = seasonal_decompose(model_data['rides'], model = "additive",period = 52)

# fig, (ax1,ax2,ax3) = plt.subplots(3,1, figsize=(15,8))
# res.trend.plot(ax=ax1,ylabel = "trend")
# res.resid.plot(ax=ax2,ylabel = "seasoanlity")
# res.seasonal.plot(ax=ax3,ylabel = "residual")
# plt.show()

### AR(1)

In [ ]:
# |L1| < 1 implies that the process is integrated of order zero, I(0)
# Differencing will allow us to use OLS, but it's unclear if we need to difference or not,
# because they also say you need to difference if L1 is close to 1 like > .9 or > .8
# But they later say to obtain the first order autocrrelation AFTER de-trending.
ar1 = AutoReg(model_data['rides'], 1).fit()
print(ar1.summary())


In [ ]:
plot_data = pd.concat([model_data, pd.Series(ar1.predict(),name='AR1')], axis=1) \
    .melt(id_vars='yearweek', value_vars=['rides','AR1'])
fig = px.line(plot_data, x='yearweek', y='value', color='variable', title='Observed vs Predicted')
fig.update_layout(xaxis=dict(type='category'))

In [ ]:
# The AR1 model actually seems to do very well in explaining the variance.
# But you can see there is a ~52 week periodicity in the residuals.
fig = plt.figure(figsize=(16, 9)) 
fig = ar1.plot_diagnostics(fig=fig, lags=53)

In [ ]:
# This shows why them model is well-fit by AR-1. 
# The first order difference is mean-zero, finite variance.
model_data_delta = model_data.assign(
    delta = model_data['rides'].diff(),
    pct_delta = lambda x: x['delta']/x['rides'])

# Drop first row, which has nothign to difference against.
model_data_delta = model_data_delta.iloc[1:].reset_index(drop=True)
print(f"mean: {model_data_delta['pct_delta'].mean()}, var: {model_data_delta['pct_delta'].var()}")
fig = px.line(model_data_delta, x='yearweek', y='pct_delta', title='first difference of Y')
fig.update_layout(xaxis=dict(type='category'))

### Overfitting??

I'm trying to wrap my head around how good this model is statistically. Recall when we call predict we use:

$$Y_t = \alpha + \rho_1 Y_{t-1} + e_t$$

When we call predict on the training set, it uses the actual observed values each time
and then applies some noise, right? Of course the fit is going to be good, because
it only applies a small deviation to the previous week's value. And since the 
dataset itself doesn't have large fluctuations independent of the previous value,
this model suits well. This quickly becomes terrible for prediction though, 
because obviously the true process is not that simple.

In [ ]:
train_size = int(len(model_data) * .5)
train, test = model_data[:train_size]['rides'], model_data[train_size:]['rides']

ar1_train = sm.tsa.AutoReg(train, lags=1)
ar1_train_fit = ar1_train.fit()

ar1_pred_train = ar1_train_fit.predict(0, len(train)-1)
ar1_pred_test = ar1_train_fit.predict(start=len(train), end=len(train)+len(test)-1)

plot_data = pd.concat([model_data, pd.Series(pd.concat([ar1_pred_train, ar1_pred_test]),name='pred')], axis=1)

plot_data = plot_data.melt(id_vars=['yearweek'], value_vars=['rides','pred'])
fig = px.line(plot_data, x='yearweek', y='value', color='variable')
fig.add_vline(x=train_size, line_dash="dash", line_color="gray")
fig.update_layout(xaxis=dict(type='category'))
fig.show()

So it sounds like (from Claude) that I should fit the model on the whole dataset,
and that the correlation-gram plot will tell me if it really is auto-regressive,
and that I should check it is stationary (which the mean is and the variance is,
except the variance has a periodicity that I should add to the model), and
that the residuals aren't auto-regressive.

### AR(1) + Holiday

In [105]:
model_data['is_holiday'] = model_data.yearweek.str.extract(r"-(\d{2})").astype(int)
model_data['is_holiday'] = model_data['is_holiday'].isin([1,47,51,52])

In [ ]:
ar1h = AutoReg(model_data['rides'], 1, exog=model_data['is_holiday']).fit()
print(ar1h.summary())

In [ ]:
plot_data = pd.concat([model_data, 
                       pd.Series(ar1.predict(),name='AR1'),
                       pd.Series(ar1h.predict(),name='AR1H')], axis=1) \
    .melt(id_vars='yearweek', value_vars=['rides','AR1','AR1H'])
fig = px.line(plot_data, x='yearweek', y='value', color='variable', title='Observed vs Predicted')
fig.update_layout(xaxis=dict(type='category'))

In [ ]:
fig = plt.figure(figsize=(16, 9)) 
fig = ar1h.plot_diagnostics(fig=fig, lags=53)

#### Check Residual

In [ ]:
resid_data = pd.concat([model_data, pd.Series(ar1h.resid, name='resid')], axis=1).dropna(how='any')
ar1hr = AutoReg(resid_data['resid'],1).fit()
print(ar1hr.summary())

In [ ]:
# It looks like there is still auto correlation in the residuals. 
fig = plt.figure(figsize=(16, 9)) 
fig = ar1hr.plot_diagnostics(fig=fig, lags=53)

In [ ]:
# Not exactly what I expect, but shows that maybe the model is under-specified.
plot_data = sm.stats.acorr_ljungbox(ar1h.resid, lags=52, return_df=True)
plot_data['significant'] = plot_data['lb_pvalue'] < .1
px.bar(plot_data, y='lb_stat', color='significant')

### AR(2)

In [ ]:
ar2 = AutoReg(model_data['rides'], 2, exog=model_data['is_holiday']).fit()
print(ar2.summary())

In [ ]:
plot_data = pd.concat([model_data, 
                       pd.Series(ar1h.predict(),name='AR1H'),
                       pd.Series(ar2.predict(),name='AR2')], axis=1) \
    .melt(id_vars='yearweek', value_vars=['rides','AR1H','AR2'])
fig = px.line(plot_data, x='yearweek', y='value', color='variable', title='Observed vs Predicted')
fig.update_layout(xaxis=dict(type='category'))

In [ ]:
fig = plt.figure(figsize=(16, 9)) 
fig = ar2.plot_diagnostics(fig=fig, lags=53)

In [ ]:
# Not exactly what I expect, but shows that maybe the model is under-specified.
plot_data = sm.stats.acorr_ljungbox(ar2.resid, lags=52, return_df=True)
plot_data['significant'] = plot_data['lb_pvalue'] < .1
px.bar(plot_data, y='lb_stat', color='significant')

In [ ]:
# The AIC and BIC indicate that AR2 > AR1H > AR1
{"AR1": (ar1.bic, ar1.aic),
 "AR1H": (ar1h.bic, ar1h.aic),
 "AR2": (ar2.bic, ar2.aic)}

### AR(2) Delta

In [ ]:
# Model seems to require 0-indexed data
model_data_delta = model_data.assign(
    delta_rides = model_data['rides'].diff()
).iloc[1:].reset_index(drop=True)
ar2d = AutoReg(model_data_delta['delta_rides'], 2, exog=model_data_delta['is_holiday']).fit()
print(ar2d.summary())

The AIC and BIC are even lower here. Theoretically I'm not sure if we want to model Y or delta_Y.

Also at this point I'm thinkink the estimates for DNC will have a lot of variance because we only have one observation
for the DNC=1 days. 

According to Claude, I should try:

* Power analysis: like R's `pwr`
* Variance Inflation Factor: which tests multicolinearity which is exacerabated by small samples (I guess because you dont have enough variance)
* Rule of thumb: for logistic regression is 10 obs per value
* Leave one out cross validation: tests for sensitivity to sample values. use R's `DFBeta` function which tells you how much the coefficient changes when you leave out each observation.


### AR(1) Delta (deprecated)

In [ ]:
# The coefs are smaller and changed sign now and the AR coef is bigger though.
# I'm not sure if this is better or worse.
delta_ar1 = AutoReg(model_data_delta['delta'], 1)
delta_ar1 = delta_ar1.fit()
print(delta_ar1.summary())

In [ ]:
# Its of course not going to do much much better because theres only randomness to explain now.
plot_data = pd.concat([model_data_delta, pd.Series(delta_ar1.predict(),name='DAR1')], axis=1) \
    .melt(id_vars='yearweek', value_vars=['delta','DAR1'])
px.line(plot_data, x='yearweek', y='value', color='variable', title='Observed vs Predicted')


In [ ]:
# This model is still ok but the residuals are slightly bigger than before.
fig = plt.figure(figsize=(16, 9)) 
fig = delta_ar1.plot_diagnostics(fig=fig, lags=53)

### AR(2)T

Woldridge suggests adding a time trend:

$$ Y_t = \vec \beta [1, X_t, t] + u_t$$

this helps mitigate spurious correlations along time, because you allow Y to vary with time directly, 
meaning your X's now have to affect Y independent of time.

Woldridge also says that this model is equivalent to de-trending the data:
ie. if you first regressed $y \sim t$ then the residuals as $\hat y \sim x$
you recover the exact same $\hat \beta$. Therefore there is no need to explicitly difference Y.

Woldridge also says it's good to include $\beta t$ when any $X$ is trending, even if $y_t$ isn't.
Because the variation of $x_t$ about its trend might still affect $y_t$, but this variation might be
small compared to the trend. Estimating the trend partials out $\beta x_t$ allowing you to estimate it
but other wise $\beta x_t$ will be swamped by neutralizing the trend part.

In [ ]:
ar2t = AutoReg(model_data['rides'], 2, exog=model_data['is_holiday'], trend='ct').fit()
print(ar2t.summary())

In [ ]:
plot_data = pd.concat([model_data, 
                       pd.Series(ar2.predict(),name='AR2'),
                       pd.Series(ar2t.predict(),name='AR2T')], axis=1) \
    .melt(id_vars='yearweek', value_vars=['rides','AR2','AR2T'])
fig = px.line(plot_data, x='yearweek', y='value', color='variable', title='Observed vs Predicted')
fig.update_layout(xaxis=dict(type='category'))

In [ ]:
fig = plt.figure(figsize=(16, 9)) 
fig = ar2t.plot_diagnostics(fig=fig, lags=53)

In [ ]:
# The AIC and BIC indicate that AR2T > AR2
{"AR2": (ar2.bic, ar2.aic),
 "AR2T": (ar2t.bic, ar2t.aic)}

### Seasonality

Side note: Woldridge says that lots of published (nominally seasonal) data is 
already seasonally adjusted (de-seasoned) as standard practice. eg GDP.

You're allowed to include seasonal and trend variables, because the seasonal 
variable operates at a higher frequency. eg sales within are dominated by seasonality 
while sales across a decade are dominated by time.



In [153]:
model_data['season'] = model_data['yearweek'].str.extract(r'-(\d{2})').astype(int)
model_data['season'] = model_data['season'] // 14

In [ ]:
# It looks like we dont need a seasonality term.
ar2s = AutoReg(model_data['rides'], 2, exog=model_data[['is_holiday','season']].astype(int), trend='ct').fit()
print(ar2s.summary())

## Event

In [ ]:
# It doesn't look like there's any noticeable spike in transit due to the concert.

DNC_ISOC_START = datetime.datetime.strptime(DNC_START, YMD).isocalendar()
DNC_ISOC_END = datetime.datetime.strptime(DNC_END, YMD).isocalendar()

fig = px.line(model_data, x='yearweek', y='rides')
fig.add_vline(x="{}-{}".format(DNC_ISOC_START.year, DNC_ISOC_START.week), line_dash="dash", line_color="gray")
fig.add_vline(x="{}-{}".format(DNC_ISOC_END.year, DNC_ISOC_END.week), line_dash="dash", line_color="gray")
fig.update_layout(xaxis=dict(type='category'))


Following Woldridge 12.2, we would want to test to show there is not serial correlation of the error terms
to be confident in our model.


# Threats

* Selection bias, obviously!
    * DNC attendees in Chicago is EXOG
        * But we know beforehand they will be concentrated in certain areas.
        * We can consider the UC to be the treatment area. 
            * But there is spillover (SUTVA violation!):
                * To airports
                * To adjacent sites
                * To the city more dispersedly
    * Imbalanced size of treatment and control groups:
        * DNC only happens during one week
        * potentially DNC only affects small parts of city
    * Selection on observables
        * Can we pick better controls via matching?
    * Exogeneity of treatment
        * Treatment should be assigned independent of potential outcomes
        * otoh the UC and MP are transit-accessible by design!
        * => this should inflate the UC coefs
* Independent panel units
    * Train, bus, and divvy stops might be co-located. How should we think of this?
        * e.g. if you take multiple transit legs so co-located ridership are correlated.
    * Spatial auto-correlation of rides across stops:
        * some baseline % of people take multiple lines, so some % of rides are correlated with additional rides some distance away
* Stable composition
    * The Damen Green Line opened a week before the event
        * Might be able to compare an after-period to parse out the contribution of this stop.
* Anticipation effects
    * The whole city prepared for years to implement this plan, so they probably did increase transit capacity both systemwide during the leadup to the treatment period and especially at UC and MP. (e.g. the Damen Green Line opened the week before)

## Potential outcomes

If we frame this as a potential outcomes, we would say:

$$
outcome =
\begin{cases} 
    \text{DNC rides} & \text{if} & \text{DNC affected} \\
    \text{non-DNC rides} & \text{if} & \text{not DNC affected} \\
\end{cases}
$$

The naive comparison would look like:
$$
\text{observed difference in rides} = \text{ATE on treated} + \text{selection bias}
$$
$$
\begin{aligned}
E[\text{rides}_i | \text{DNC}_i = 1] - E[\text{rides}_i | \text{DNC}_i = 0]
&= E[\text{rides}_{1i} | \text{DNC}_i = 1] - E[\text{rides}_{0i} | \text{DNC}_i = 1] \\
&+ E[\text{rides}_{0i} | \text{DNC}_i = 1] - E[\text{rides}_{0i} | \text{DNC}_i = 0] \\
\end{aligned}
$$
The ATE on treated is the difference in potential outcomes among those actually selected for treatment.
So in this case it is the difference in rides during the DNC vs the counterfactual rides if the DNC didn't happen.

The selection bias is the difference in counterfactual rides if the DNC didn't happen vs observed rides not affected by DNC.
I'm not actually sure if this would be positive or negative. However the UC generally has a lot of rides, I think,
so I'm guessing a positive selection bias?



## Conditional Independence Assumption

The CIA asserts that Y is independent of D when you condition on X ie. $E[Y | X,D] = E[Y|X]$. 

It implies that selection bias disappears once you condition on X. This obviously isn't always true. It's mostly used to  motivate textbook naive estimators.

In an observational study it asserts that D is as-good-as-random.

# Control Group

TODO!

Decide whether to keep all stations as controls or to only keep a subset. 
Remember a control should be comparable, so it can't just be a random sample of other routes.
Maybe other central routes?

Maybe use some kind of quantile/matching procedure? How should that work should
they be similar on facts or outcomes prior to the treatment period? Like should
we take all the stations with total ridership similar to this one in the years prior?

TODO!

Also decide what other controls to add. Like maybe distance from city center? Or neighborhood?

TODO!

How would we construct Ubers as a panel? Since they are point-to-point, how should we spatially aggregate them?

* We could snap them to the current panel? 
    * But I'm sure most points don't intersect a station. That's kind of the point of rideshares.
* We could snap them to the nearest panel?
* We could aggregate the panel to census tracts?
    * This would ameliorate the stop co-location issue.
    * And would significantly decrease the data size. Though bus routes would be difficult to map.
* *Actually the Uber data is anonymized to Census Tract or Community Area level!*
* It's also HUGE. 144M ROWS!

TODO!
Read over Woldridge Ch 13 and 14 to see if panel data makes sense in this context.
Note that we only have one observation still, so do we have enough variation since
we might need to use clustered errors. And if we are at the neighborhood level
we might only have one member of the treated panel.

# Model Design

I'm not sure if it applies here but I wanted to know how to basically run separate regressions per covariate value. For this example we had train and bus data and I wanted a separate intercept and slope of DNC for train and bus. 

But I wasn't sure what the correct way to do this is. Do you just run `rides ~ train*DNC` or  do you need to run two separate regressions? 

This took me into Mixed Linear Effects models which you seem to run when you have non-independence between observed units, for example if you have multiple obsevations per subject where each subject has its own idiosyncratic intercept, or like each survey question might have its own idiosyncratic mean and or slope. (You can choose if you want to model just random intercepts or if you want to interact the interesting regressors with the grouping column to get group-specific slopes. Some authors advise to "keep it maximal" in an actual controlled experiment and to use all covariates justified by the design.) The idea is you can explicitly model the between group variation, which helps give you more precise estimators on your other regressors. But you don't actually care about the individual group means -- I think it models it but doesn't report it. 

This method uses the terminiology "fixed effect" for the regressors you care about and "random effect" for the regressors that indicate these groups. This is a confusing terminology because it sounds like a "fixed effect" model where you add a dummy variable per group. However the MLEM random effects are a deviation from the sample average, and again you don't report the group coefficients, just the variance of the coefs. Proper fixed effects models report the per group coefficient. The MLEM random effects basically model the grouping column as a normal distribution where each group's mean is sampled from this distribution. This allows you to model hundreds/thousands of groups. The "fixed effects" model doesn't assume the groups have a common distribution. And since it estimates an actual coefficient per group, you can't have high-cardinality groups. 

MLEM uses "partial pooling" where it estimates the group-level intercepts as a variance-weighted average of the group vs population means. 

$$
\hat b_j = \frac{\sigma^2_{pop}}{\sigma^2_{pop} + \sigma^2_j / n_j}(\bar y_j - \bar y_{pop})
$$

so when $n_j$ (group size) is large, the weight approaches unity so the estimate tends
towards the group deviation from the population: $\hat b_j \to (\bar y_j - \bar y_{pop})$
otoh when $n_j$ is small, the $\sigma^2_j$ pulls the overall weight to zero, making 
$\hat b_j \to 0$ meaning no deviation from the population mean.

This shows how MLEM is different from a true "fixed effects" model because it incorporates info from all groups, while the fixed effects models estimates each (j-1) group independently (vs a baseline/reference group). Crucially the FE doesn't borrow strength from other groups: it takes the group differences as-is which makes it prone to high variance in its coefficients estimates particularly in groups with few observations.

I was also researching if this is different from clustered standard errors. Clustered 
standard errors don't actually estimate separate coefficients per group. Instead they
correct the standard errors on the population coefficients to account for non-independence
(ie correlation in the errors) via grouping. So basically I guess it makes your
standard errors bigger because you have less independent information than you think you have.
This method applies to situations where you don't care about group-specific means
because you don't actually think the means necessarily are different across groups,
but you do think the residuals are correlated per group.

We might be able to model this as

$$
\text{rides}_{it} = \alpha_0 + \beta_0 \text{rides}_{i,t-1} + \beta_1 t + \gamma_0 (1|i) + \gamma_i +
\delta_0 \text{DNC}_t + \delta_1 \text{perimeter}_i + \delta_2 \text{DNC}_t \text{perimeter}_i + u_{it}
$$

So as a DiD model ($\delta$) and optionally an AR(1) process ($\beta$) and optionally a MLEM ($\gamma$) or a fixed effect ($\gamma$).

# More Panel Models

TODO!

I have a lot of notes in this section! 

Unfortunately I think Woldridge p. 461 says that the assumptions rule out allowing
any X's to be a lagged Y ie $x_{it} = y_{i,t-1}$ because that would allow $\Delta u_i$ to correlate
with $\Delta X_i$. 

Also $\Delta X_i$ kind of gets rid of per-unit levels of explanatory variables
so it reduces variation in X if the timelike variation isn't so impressive.

For example: $\text{income}_{it} \sim \text{education}_{it}$ yields $\Delta \text{income}_{i} \sim \Delta \text{education}_i$
but even though the panel might have a lot of variation education, it might not have 
lots of variation in *changes* in education (like once you're an adult your education doesn't change as much).

Woldridge (p. 467) notes that the differenced estimator is the two-period panel version of DiD. But it has the advantage of being able to control for time invariant unit effects.

We're allowed to have dummy intercepts per time period in this framework. ie

$$ y_{it} = ... + \delta_1 \mathbb{1}_{t=1} + \delta_2 \mathbb{1}_{t=2} + ...$$

this is preferred when T is small, rather than having an actual time covariate $\beta t$
which is less flexible.

For convenience, t=3 and onwards instead of using $\delta_2 \Delta \mathbb{1}_{t=2}$ etc.
because those delta indicators flip from 0 to 1 to -1 to 0 whereas the dummies are simply just 0 1 0.

These models require $Cov(x_{itj}, u_{it}) = 0$ which is broken if $x_{itj}$ is a lagged dependent variable, especially if it is auto-regressive. Having more time periods does not solve this.

ICBST if $u_{it}$ is uncorrelated over time, then $\Delta u_{it}$ and $\Delta u_{i,t+1}$ are correlated as -.5 such as when $u_{it}$ is AR(1) but if $u_{it}$ is a random walk then it is ok. We can test for this by getting the residuals on the un-differenced model, differencing to compute $r_{it} = \Delta u_{it}$, and running simple OLS (NOT an actual AR model) as $r_{it} = \rho r_{i,t-1} + e_{it}$ and testing $H_0: \rho = 0$.

Woldridge (p. 474) says FD can be really bad if you have measurement error, because differencing reduces X's variation relative to its error.

TODO! Continue reading Woldridge chapter 14.

TODO! Incorporate airports as quasi-treatment parameters